This notebook aims to assess the generalization accuracy of a generated suffix, assuming a data-split was used during training.

In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import torch
from torch import nn
import matplotlib.pyplot as plt
from copy import deepcopy
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
from transformers import AutoTokenizer
import pandas as pd
import seaborn as sns
from types import SimpleNamespace
from datasets import Dataset
from os.path import join as oj
import pickle as pkl
import os
import dvu
dvu.set_style()
import analyze_utils
import sys
sys.path.append('..')
import data
from model_utils import prompt_classification

In [13]:
dir_suffix_search = '/home/chansingh/mntv1/suffix_long_rerun'
r_suffix_search = analyze_utils.load_results_and_cache(dir_suffix_search, save_file='r.pkl', only_keep_scalar=False)
r_suffix_search = pd.read_pickle(os.path.join(dir_suffix_search, 'r.pkl'))
r_suffix_search = analyze_utils.postprocess_results(r_suffix_search,   )
r = r_suffix_search
r.to_pickle('../results/prompt_gen/results_suffix_all.pkl')
r = pd.read_pickle('../results/prompt_gen/results_suffix_all.pkl')

 36%|███▋      | 4/11 [00:00<00:00, 14.02it/s]

skipping 1729944cd6bc0e3ef9f2bd0f993f446299bc37d234ab8ec1cd10adf654fe336bnnec


 64%|██████▎   | 7/11 [00:41<00:26,  6.67s/it]

skipping 250b0795aa1ce367496b264e054ff581ddc6b2646f1c624a84dd4e8b1e34a19fezyo


100%|██████████| 11/11 [00:41<00:00,  3.81s/it]


skipping f006c327d847865202be965fd0073d7c6fa071b056958908db2f79a7ffba3c71ijqk


# Look at top suffix candidates

In [22]:
r0 = r[(r.max_num_tokens == 6) * (r.n_shots == 1) * (r.checkpoint == 'EleutherAI/gpt-j-6B')]
print(r0.shape)
print(list(r0.keys()))

(0, 56)
['task_name', 'n_shots', 'max_dset_size', 'max_digit', 'template_num_init_string', 'template_num_task_phrasing', 'train_split_frac', 'checkpoint', 'max_num_tokens', 'beam_size', 'beam_size_extra', 'use_single_query', 'use_stopwords', 'use_early_stopping', 'use_generic_query', 'float16', 'seed', 'save_dir', 'use_cpu_only', 'use_parallelformers', 'use_cache', 'use_verbose_saving', 'epoch_save_interval', 'batch_size', 'task_name_list', 'suffix_str_init', 'len_suffix_str_init', 'suffix_str_added', 'num_tokens_added', 'num_model_queries', 'running_prob', 'final_answer_pos_initial_token', 'final_answer_full', 'final_answer_added', 'final_model_queries', 'final_num_suffixes_checked', 'final_answer_depth', 'top_prompt', 'top_candidates', 'top_probs', 'final_answer_found', 'Recall @ 3 suffixes', 'Recall @ 5 suffixes', 'Recall @ 10 suffixes', 'Recall @ 15 suffixes', 'Recall @ 20 suffixes', 'Recall @ 25 suffixes', 'Recall @ 30 suffixes', 'Recall @ 40 suffixes', 'Recall @ 50 suffixes', 'Re

The most relevant keys are top_prompt, top_candidates, top_probs.

The relevant keys for metrics are reciprocal_rank, reciprocal_rank_multi.


**Save out top prompts**

In [25]:
results_acc_dir = oj('/home/chansingh/interpretable-autoprompting', 'results', 'generalization_acc')
prompts = pkl.load(open(oj(results_acc_dir, 'prompts_all.pkl'), 'rb'))

In [37]:
idxs = prompts.index
def get_top_prompt(r0, task_name):
    top_prompt = r0[r0['task_name'] == task_name]['top_prompt']
    if len(top_prompt) == 0:
        return np.nan
    else:
        return top_prompt
prompts_suff = [get_top_prompt(r0, task_name) for task_name in idxs]
# prompts['suffix'] = 